# Getting started with OGGM:3D-Viz

This notebook gives you a quick overview of how to get started with OGGM:3D-Viz for creating 3D visualizations of dynamic glacier runs. Although it is designed to work seamlessly with native OGGM outputs, you can use any `xarray.Dataset` as long as it follows the data format guidelines described below.

## Open input data for visualization

OGGM-3DViz is designed to work seamlessly with output generated by OGGM. However, if you want to create 3D glacier visualizations from purely measurements or other dynamic glacier models, you can do so. You only need to structure the NetCDF input data accordingly. Let's have a look at an example (created and copied from this [OGGM tutorial](https://oggm.org/tutorials/stable/notebooks/tutorials/distribute_flowline.html)):

In [ ]:
import xarray as xr

ds_glacier = xr.open_dataset("constant_climate_dummy_data.nc")
ds_glacier

You see the file has some coordinates and data variables, not all of them are needed for a minimum visualisation, but some of them are nice features for addons, shown at other points of the tutorials. The minimum data needed for the visualisation are the coordinates (`x`, `y`, `time`) and the data variables (`bedrock`, `distributed_thickness`). However, you can decide on the naming as they can be explicitly set during the creation of the 3D visualization object in the next step.

## Create the 3D visualization object

Creating a 3D visualization object is quite easy. Just provide your data and define the names of the coordinates and data variables which should be used. The default variables are the ones used by OGGM (see example data above), but for demonstration, we set everything explicitly here:

In [ ]:
from oggm_3dviz.tools import Glacier3DViz

viz = Glacier3DViz(
    ds_glacier,  # dataset for visualization
    x='x',  # x-coordinate name, default is 'x'
    y='y',  # y-coordinate name, default is 'y'
    time='time',  # time coordinate name, default is 'time'
    topo_bedrock='bedrock',  # bedrock data variable, default is 'bedrock'
    ice_thickness='simulated_thickness',  # evolving ice thickness variable, default is 'simulated_thickness'
    time_var_display='calendar_year',  # time format displayed, opting for total years here
    camera_args={  # camera view settings, more details later
        'azimuth': -135, # rotate around the z-axis
        'elevation': 20,  # elevation angle above ground
    }
)

And thats all! Now lets see what we can do with the `viz`:

## Interactive visualization in notebook

Opening an interactive visualization in the notebook and explore your data is straightforward. However, for this to work you must make sure that `ipywidgets` is correctly installed ([https://ipywidgets.readthedocs.io/en/latest/user_install.html](https://ipywidgets.readthedocs.io/en/latest/user_install.html)).

In the interactive visualization, you can use the slider to scroll through time, you can zoom and change the point of view, and finally, you can directly save it to an image.

In [ ]:
viz.show()

## Export animation

Or you can easily export it as an animation:

In [ ]:
viz.export_animation(filename='dummy_animation.mp4',  # filename for the animation, should end with .mp4
                     framerate=20,  # framerate of the animation, affects duration and speed
                     quality=5,  # The range is 0 - 10. Higher quality leads to a larger file.
                    )


## Plot a single year

Or extract some static plots of single years for static visualizations:

In [ ]:
viz.plot_year(2050,  # year to visualize
              filepath='dummy_fig_2050.png',  # file path for saving the image; if not provided, displays in notebook only
              show_plot=True,  # whether to show the plot in the notebook when saved
             )

## Enriching the visualization

Creating initial visualizations with OGGM:3D-Viz is straightforward, but you might want to enhance these visuals by adapting the topography texture or adding annotations. For instance, using satellite imagery for the topography or annotating geographic features can significantly improve the visual impact. Below is an example that demonstrates how to customize your visualization with various annotations and textures:

In [ ]:
from oggm_3dviz.tools.map_annotations import PointAnnotation, ArrowAnnotation, MaskAnnotation, LegendAnnotation
from oggm_3dviz.tools import Glacier3DViz

# Define annotations for the visualization
north_arrow = ArrowAnnotation(text='N', y_position=0.3, x_position=1.1, arrow_magnitude=0.3,
                              text_position_offset=[-0.02, 0.02, 0],)
finsteraarhorn = PointAnnotation(latitude=46.53703, longitude=8.12611, height=4274,
                                 text='Finsteraarhorn\n    4274 m',)
outline_area = MaskAnnotation(mask_data='glacier_mask', mask_color=[255, 0, 0, 100],)
outline_legend = LegendAnnotation(labels=[[' RGI Outlines', [255, 0, 0, 100]]],)

# Create a 3D visualization object with enhanced settings
viz_adapted_example = Glacier3DViz(
    ds_glacier.isel(  # select a subset of the dataset for better focus
        x=slice(120, 355),
        y=slice(120, 340)),
    plotter_args={'window_size': [650, 400]},  # customize the window size for the plot
    add_mesh_ice_thick_args={'scalar_bar_args': {'position_x': 0.05}},  # adjust scalar bar positioning
    use_texture=True,  # utilize background map (satellite imagery) for topography texture
    camera_args={  # fine-tune the camera settings
        'zoom': 1.3,
        'azimuth': -135,  # rotation around the z-axis
        'elevation': 20,  # elevation angle above the ground
    },
    additional_annotations=[north_arrow, finsteraarhorn, outline_area, outline_legend],  # apply annotations
)

# Display the visualization for the year 2040
viz_adapted_example.plot_year(2040)

For more in-depth explanations on how to adapt and refine your visualizations as shown above, please refer to the following tutorials:
- [General styling of 3D visualizations](general_styling.ipynb)
- [Annotating your 3D glacier visualizations](annotations.ipynb)